###

In [22]:
import psycopg2
import pandas as pd
import db

In [23]:
# Establish connection to the database
conn = psycopg2.connect(
    dbname="nyc_citibike",
    user=db.user,
    password=db.password,
    host="localhost",
    port="5432"
)
# Create a cursor
cur = conn.cursor()

In [3]:
# List of table names
dates = [*range(2209,2213)] + [*range(2301,2313)] + [*range(2401,2409)]
tables = [f'rides{x}' for x in dates]

In [7]:
# Perform cleaning operations on each table
for table in tables:
    for x in ['start','end']:
        # remove trailing _
        query_ = f"""UPDATE {table} SET {x}_station_id = RTRIM({x}_station_id, '_');"""
        cur.execute(query_)
        # append 0 to station indexes that don't have two trailing numbers after .
        query0 = f"""UPDATE {table} 
            SET {x}_station_id = REGEXP_REPLACE({x}_station_id, '^(\d+\.\d)$', '\1' || '0')
            WHERE {x}_station_id ~ '^\d+\.\d$';"""
        cur.execute(query0)
        # correct for inconsistent lat lng for stations
        queryx = f"""
            WITH most_frequent_lat_lng AS (
                SELECT {x}_station_name,
                       {x}_lat, 
                       {x}_lng, 
                       COUNT(*) AS freq
                FROM {table}
                GROUP BY {x}_station_name, {x}_lat, {x}_lng
            ),
            most_frequent AS (
                SELECT {x}_station_name, {x}_lat AS correct_lat, {x}_lng AS correct_lng
                FROM most_frequent_lat_lng
                WHERE ({x}_station_name, freq) IN (
                    SELECT {x}_station_name, MAX(freq)
                    FROM most_frequent_lat_lng
                    GROUP BY {x}_station_name
                )
            )
            UPDATE {table}
            SET {x}_lat = most_frequent.correct_lat,
                {x}_lng = most_frequent.correct_lng
            FROM most_frequent
            WHERE {table}.{x}_station_name = most_frequent.{x}_station_name
            AND ({table}.{x}_lat <> most_frequent.correct_lat OR {table}.{x}_lng <> most_frequent.correct_lng);
        """
        cur.execute(queryx)

In [17]:
# Rollback if there is an error
conn.rollback()

In [8]:
# Commit the changes
conn.commit()

In [13]:
# Perform data transformations and export to CSVs
for table in tables:
    query = f"""
        WITH cleaned AS (
            SELECT * FROM {table}
            WHERE start_station_name IS NOT NULL
            AND end_station_name IS NOT NULL
            AND start_station_id ~ '^\d{{4}}\.\d{{2}}$'
            AND end_station_id ~ '^\d{{4}}\.\d{{2}}$'
            AND start_lat IS NOT NULL
            AND end_lat IS NOT NULL
            AND (start_lat <> end_lat OR start_lng <> end_lng)
        ),
        binned_starts AS (
            SELECT TO_CHAR(started_at, 'YYYYMM')::INTEGER AS date,
            start_station_name AS name, 
            start_lat AS lat, start_lng AS lng,
            CASE WHEN EXTRACT(DOW FROM started_at) IN (0, 6) THEN 'Weekend' ELSE 'Weekday' END AS day, 
            EXTRACT(HOUR FROM started_at) AS hour,
            AVG((end_lat-start_lat)/SQRT(POWER(end_lat-start_lat,2)+POWER(end_lng-start_lng,2))) as i,
            AVG((end_lng-start_lng)/SQRT(POWER(end_lat-start_lat,2)+POWER(end_lng-start_lng,2))) as j,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY earth_distance(ll_to_earth(start_lat, start_lng), ll_to_earth(end_lat, end_lng))) AS distance,
            COUNT(*) as counts
            FROM cleaned
            GROUP BY date, name, lat, lng, day, hour
            ORDER BY date, name, lat, lng, day, hour
        ),
        binned_ends AS (
            SELECT TO_CHAR(ended_at, 'YYYYMM')::INTEGER AS date,
            end_station_name AS name,
            CASE WHEN EXTRACT(DOW FROM ended_at) IN (0, 6) THEN 'Weekend' ELSE 'Weekday' END AS day, 
            EXTRACT(HOUR FROM ended_at) as hour,
            AVG((end_lat-start_lat)/SQRT(POWER(end_lat-start_lat,2)+POWER(end_lng-start_lng,2))) as i,
            AVG((end_lng-start_lng)/SQRT(POWER(end_lat-start_lat,2)+POWER(end_lng-start_lng,2))) as j,
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY earth_distance(ll_to_earth(start_lat, start_lng), ll_to_earth(end_lat, end_lng))) AS distance,
            COUNT(*) as counts
            FROM cleaned
            GROUP BY date, name, day, hour
            ORDER BY date, name, day, hour
        )
        SELECT s.date, s.name, s.lat, s.lng, s.day, s.hour,
        s.i as s_i, s.j as s_j, s.distance as s_median_dist, s.counts as s_count,
        e.i as e_i, e.j as e_j, e.distance as e_median_dist, e.counts as e_count
        FROM binned_starts AS s
        JOIN binned_ends AS e
            ON s.date = e.date AND s.name = e.name AND s.day = e.day AND s.hour = e.hour
        ORDER BY s.date, s.name, s.day, s.hour
    """
    cur.execute(query)
    # Fetch the result as a DataFrame
    columns = [desc[0] for desc in cur.description]  # Get column names
    result = cur.fetchall()  # Fetch all data
    # Convert to pandas DataFrame
    df = pd.DataFrame(result, columns=columns)
    # Export to CSV
    df.to_csv(f'data/20{table[5:9]}_citibike_tripdata.csv', index=False)
    print(table)

rides2209
rides2210
rides2211
rides2212
rides2301
rides2302
rides2303
rides2304
rides2305
rides2306
rides2307
rides2308
rides2309
rides2310
rides2311
rides2312
rides2401
rides2402
rides2403
rides2404
rides2405
rides2406
rides2407
rides2408


In [20]:
# Commit the changes and close the connection
cur.close()
conn.close()